In [64]:
# Imports
import numpy as np
import csv
import sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

# Predict via the user-specific median.
# If the user has no data, use the global median.

# Hard-code file names
train_file = 'data/LoanStats_2016Q1.csv'

In [65]:
df = pd.read_csv(train_file)
print df.shape

(133889, 111)


In [66]:
feature_set = ['loan_amnt', 'installment', 'annual_inc']
features = df[feature_set]

In [67]:
results = df['loan_status']
loan_statuses = list(set(results))
cleaned_results = map(lambda result: loan_statuses.index(result), results)

# Add cleaned_results back to features data frame
features['cleaned_results'] = cleaned_results

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [80]:
features['cleaned_results'].value_counts()

3    127145
2      4431
6      1093
1       801
4       359
5        58
Name: cleaned_results, dtype: int64

In [68]:
# count number of null rows
print np.sum(pd.isnull(features))
features = features.dropna(axis=0)

loan_amnt          2
installment        2
annual_inc         2
cleaned_results    0
dtype: int64


In [69]:
X = features.iloc[:,:-1]
y = features.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=37)

In [70]:
regressor = RandomForestClassifier(n_estimators=150, min_samples_split=1)
regressor.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=1,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
# regressor.predict(X_test)

print "Testing error - RandomForest Score was: %f" % (regressor.score(X_test, y_test))
print "Training error - RandomForest Score was: %f" % (regressor.score(X_train, y_train))

Testing error - RandomForest Score was: 0.941818
Training error - RandomForest Score was: 0.984885
